In [1]:
import requests 
import time
import pandas as pd
import numpy as np
from config import stocks_api

In [2]:
nasdaq = pd.read_csv("Resources/nasdaq_screener_1650145895697.csv") # Read Nasdaq csv
notdaq = pd.read_csv("Resources/not_nasdaq.csv") # Elements that are in Nasdaq, continually appended 

nasdaq_tickers = nasdaq["Symbol"] # Create list of unique nasdaq tickers
not_nasdaq_tickers = notdaq["0"] # Create list of not nasdaq tickers
for ticker in not_nasdaq_tickers: # Drop any elements that aren't tickers
    if ticker in nasdaq_tickers:
        nasdaq_tickers.drop(ticker)

nasdaq_data = [] # Empty list to hold our dictionary
nasdaq_null = [] # List to hold null values 

time_span = "day" #time_span = input()
multiplier = "1" # time_span_multiplier = input()
time_span = "day" #time_span = input()
start_date = "2022-03-17" #start_date = input()
end_date = "2022-03-23" #end_date = input()

# Create counters.
record_count = 1
set_count = 1


In [ ]:
for ticker in nasdaq_tickers:
 
    try:
        url = "https://api.polygon.io/v2/aggs/ticker/" + ticker + "/range/" + multiplier + "/" + time_span + "/" + start_date + "/" + end_date + "?adjusted=true&sort=asc&limit=120&apiKey=" + stocks_api
        ticker_request = requests.get(url)    
        ticker_data = ticker_request.json()
        
        volume = []
        volume_weighted_average = []
        open_price = []
        close_price = []
        max_price = []
        lowest_price = []
        unix_time = []
        number_of_transactions = []

        for j in range(len(ticker_data["results"])):
            volume.append(ticker_data["results"][j]["v"])
            volume_weighted_average.append(ticker_data["results"][j]["vw"])
            open_price.append(ticker_data["results"][j]["o"])
            close_price.append(ticker_data["results"][j]["c"])
            max_price.append(ticker_data["results"][j]["h"])
            lowest_price.append(ticker_data["results"][j]["l"])
            unix_time.append(ticker_data["results"][j]["t"])
            number_of_transactions.append(ticker_data["results"][j]["n"])
    
    
    
        nasdaq_data.append({"Tickers": ticker, 
                            "Volume": volume,
                            "Volume Weighted Average": volume_weighted_average,
                            "Open Price": open_price,
                            "Close Price": close_price,
                            "Max Price": max_price,
                            "Lowest Price": lowest_price,
                            "Unix Time": unix_time,
                            "Number of Transactions": number_of_transactions})

    except:
        print(f"The Ticker, " + ticker + " is unavailible")
        nasdaq_null.append(ticker)
        nasdaq_null_df = pd.DataFrame(nasdaq_null)
        nasdaq_null_df.to_csv("Resources/not_nasdaq_new.csv", index_label="Tickers")
               
        pass
    
    time.sleep(12) # Free subscription only allows for 5 requests a minute
       

The Ticker, AAIC^B is unavailible
The Ticker, AAIC^C is unavailible
The Ticker, AAM^A is unavailible
The Ticker, AAM^B is unavailible
The Ticker, ABR^D is unavailible
The Ticker, ABR^E is unavailible
The Ticker, ABR^F is unavailible
The Ticker, ACAX is unavailible
The Ticker, ACAXR is unavailible
The Ticker, ACAXW is unavailible
The Ticker, ACP^A is unavailible
The Ticker, ACR^C is unavailible
The Ticker, ACR^D is unavailible
The Ticker, ACTDU is unavailible
The Ticker, ADC^A is unavailible
The Ticker, AEHAU is unavailible
The Ticker, AEL^A is unavailible
The Ticker, AEL^B is unavailible
The Ticker, AFARU is unavailible
The Ticker, AGM^C is unavailible
The Ticker, AGM^D is unavailible
The Ticker, AGM^E is unavailible
The Ticker, AGM^F is unavailible
The Ticker, AGM^G is unavailible
The Ticker, AHH^A is unavailible
The Ticker, AHL^C is unavailible
The Ticker, AHL^D is unavailible
The Ticker, AHL^E is unavailible
The Ticker, AHT^D is unavailible
The Ticker, AHT^F is unavailible
The Ticke

In [ ]:
nasdaq_df = pd.DataFrame(nasdaq_data)
output_data_file = "Resources/Nasdaq_data.csv"

nasdaq_df.to_csv(output_data_file, index_label="Ticker number")
nasdaq_df